In [6]:
# the service is currently broken on the backend

from io import BytesIO
import cartopy.crs as ccrs 
import numpy as np
import requests
import xarray as xr
import hvplot.xarray

def download_data(url, **kwargs):
    r = requests.get(url)
    buf = BytesIO(r.content)
    return xr.open_dataset(buf, **kwargs)    

# Generate data remotely
cmda_url = 'http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/correlationMap'
model1, var1, pres1 = 'GFDL_ESM2G', 'cli', 500
model2, var2, pres2 = 'GFDL_ESM2G', 'rlutcs', -999999
laggedTime = 0
width, height = 1000, 500
v1 = f'{model1}:{var1}'
v2 = f'{model2}:{var2}'
query = dict(
    model1=model1, 
    var1=var1, 
    pres1=pres1, 
    model2=model2, 
    var2=var2, 
    pres2=pres2, 
    lonS=0, 
    lonE=360, 
    latS=-90, 
    latE=90, 
    timeS=200401, 
    timeE=200412, 
    laggedTime=laggedTime, 
    purpose=''
)

# call the web api
r = requests.get(cmda_url, params=query)
print(r.url)
print(r.status_code)
print(r.json()['message'])

http://ec2-52-53-95-229.us-west-1.compute.amazonaws.com:8080/svc/correlationMap?model1=GFDL_ESM2G&var1=cli&pres1=500&model2=GFDL_ESM2G&var2=rlutcs&pres2=-999999&lonS=0&lonE=360&latS=-90&latE=90&timeS=200401&timeE=200412&laggedTime=0&purpose=
200
 Current size of FERRET memory cache: 100 MegaWords  (1 word = 4 bytes)
 Current size of FERRET memory cache: 100 MegaWords  (1 word = 4 bytes)
cmacDir:  /home/svc/new_github/CMDA
desDir:  /home/svc/new_github/CMDA/JPL_CMDA/services/svc/svc/src/des
configFile:  /home/svc/new_github/CMDA/JPL_CMDA/services/svc/data.cfg
temp1:  /mnt/data/data_2016

a.var1=argDict[k]
a.latS=argDict[k]
a.var2=argDict[k]
a.model2=argDict[k]
a.model1=argDict[k]
a.lonE=argDict[k]
a.timeS=argDict[k]
a.latE=argDict[k]
a.purpose=argDict[k]
a.pres2=argDict[k]
a.timeE=argDict[k]
a.pres1=argDict[k]
a.lonS=argDict[k]
a.laggedTime=argDict[k]
a.outDir=argDict[k]
variable  0
gfdl esm2g lai
/home/svc/new_github/CMDA/JPL_CMDA/services/svc/svc/src/des/gfdl_esm2g/lai.des
/mnt/data_2

In [3]:
# Download data into xarray Dataset object
data_url = r.json()['dataUrl']
ds = download_data(data_url)
print(ds.info())
ds

ValueError: <_io.BytesIO object at 0x7f9949808230> is not a valid netCDF file did you mean to pass a string for a path instead?

In [5]:
# data from api
from skimage import io
image = io.imread(r.json()['url'])
f,ax = plt.subplots(1,figsize=(20,16))
ax.imshow(image)
plt.axis('off')
plt.show()

HTTPError: HTTP Error 404: NOT FOUND

In [ ]:
def plot_data(data, **kwargs):
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_global(); ax.coastlines();
    return data.plot(ax=ax, transform=ccrs.PlateCarree(), **kwargs)
ds.patterns.hvplot.quadmesh('lon', 'lat', coastline=True)

In [ ]:
ds.tser.hvplot.line(x='time', y='tser')